# 导入库

In [1]:
from keras.layers import Input,LSTM,Dense
from keras.models import Model,load_model
from keras.utils import plot_model

import pandas as pd
import numpy as np
import re

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

In [2]:
def create_model(n_input,n_output,n_units):
    #训练阶段
    #encoder
    encoder_input = Input(shape = (None, n_input))
    #encoder输入维度n_input为每个时间步的输入xt的维度，这里是用来one-hot的英文字符数
    encoder = LSTM(n_units, return_state=True)
    #n_units为LSTM单元中每个门的神经元的个数，return_state设为True时才会返回最后时刻的状态h,c
    _,encoder_h,encoder_c = encoder(encoder_input)
    encoder_state = [encoder_h,encoder_c]
    #保留下来encoder的末状态作为decoder的初始状态
    #decoder
    decoder_input = Input(shape = (None, n_output))
    #decoder的输入维度为中文字符数
    decoder = LSTM(n_units,return_sequences=True, return_state=True)
    #训练模型时需要decoder的输出序列来与结果对比优化，故return_sequences也要设为True
    decoder_output, _, _ = decoder(decoder_input,initial_state=encoder_state)
    #在训练阶段只需要用到decoder的输出序列，不需要用最终状态h.c
    decoder_dense = Dense(n_output,activation='softmax')
    decoder_output = decoder_dense(decoder_output)
    #输出序列经过全连接层得到结果
    
    #生成的训练模型
    model = Model([encoder_input,decoder_input],decoder_output)
    #第一个参数为训练模型的输入，包含了encoder和decoder的输入，第二个参数为模型的输出，包含了decoder的输出
    
    #推理阶段，用于预测过程
    #推断模型—encoder
    encoder_infer = Model(encoder_input,encoder_state)
    
    #推断模型-decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))    
    decoder_state_input = [decoder_state_input_h, decoder_state_input_c]#上个时刻的状态h,c   
    decoder_infer_output, decoder_infer_state_h, decoder_infer_state_c = decoder(decoder_input,initial_state=decoder_state_input)
    decoder_infer_state = [decoder_infer_state_h, decoder_infer_state_c]#当前时刻得到的状态
    decoder_infer_output = decoder_dense(decoder_infer_output)#当前时刻的输出
    decoder_infer = Model([decoder_input]+decoder_state_input,[decoder_infer_output]+decoder_infer_state)
    return model, encoder_infer, decoder_infer

In [3]:
N_UNITS = 256
BATCH_SIZE = 32
EPOCH = 10
NUM_SAMPLES = 1024

## 读取数据

In [4]:
def load_data(strname, nmin, nmax):
    words = []
    y_train = []
    maxn = 0
    with open(strname, 'r') as f:
        lines = []
        i = 0
        for l in f:
            i += 1
            if i <= nmin:
                continue;
            l = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', l)
            lines.append(l)
            if i >= nmax:
                break
        for l in lines:
            sentence = l.split(' ')
            if len(sentence) > maxn:
                maxn = len(sentence)
            words.append(sentence)
    len1 = len(words)
    x_train=[]

    for i in range(len1):
        words_one = ['\t'] + words[i]
        len_word = len(words_one)
#         if len_word <= maxn:
#             for k in range(maxn-len_word):
#                 words_one.append("#")
        x_train.append(words_one)

    return x_train

In [5]:
nmin = 0
nmax = NUM_SAMPLES
input_texts = load_data('training/commoncrawl.de-en.de', nmin, nmax)
target_texts = load_data('training/commoncrawl.de-en.en', nmin, nmax)
print(input_texts[1])

print(np.array(input_texts).shape,flush=True)
print(np.array(target_texts).shape,flush=True)

['\t', 'Nach', 'der', 'Aushärtung', 'schützt', 'iron', 'cement', 'die', 'Kokille', 'gegen', 'den', 'heissen', ',', 'abrasiven', 'Stahlguss', '.', '\n']
(1024,)
(1024,)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [6]:
def train_features(z_train):
    s = set([])
    for line in z_train:
        for wd in line:
            s.add(wd)
    return sorted(list(s)) 

In [7]:
input_characters = train_features(input_texts)
target_characters = train_features(target_texts)

In [8]:
INUPT_LENGTH = max([len(i) for i in input_texts])
OUTPUT_LENGTH = max([len(i) for i in target_texts])
INPUT_FEATURE_LENGTH = len(input_characters)
OUTPUT_FEATURE_LENGTH = len(target_characters)

## 向量化

In [9]:
encoder_input = np.zeros((NUM_SAMPLES,INUPT_LENGTH,INPUT_FEATURE_LENGTH))
decoder_input = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))
decoder_output = np.zeros((NUM_SAMPLES,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))

In [10]:
input_dict = {char:index for index,char in enumerate(input_characters)}
input_dict_reverse = {index:char for index,char in enumerate(input_characters)}
target_dict = {char:index for index,char in enumerate(target_characters)}
target_dict_reverse = {index:char for index,char in enumerate(target_characters)}

In [11]:
for seq_index,seq in enumerate(input_texts):
    for char_index, char in enumerate(seq):
        encoder_input[seq_index,char_index,input_dict[char]] = 1

In [12]:
for seq_index,seq in enumerate(target_texts):
    for char_index,char in enumerate(seq):
        decoder_input[seq_index,char_index,target_dict[char]] = 1.0
        if char_index > 0:
            decoder_output[seq_index,char_index-1,target_dict[char]] = 1.0

# 验证集

In [13]:
import xml

def load_test(strname):
    words = []
    x_test = []
    maxn = 0
    dm = xml.dom.minidom.parse(strname)
    eles = dm.documentElement
    segs = eles.getElementsByTagName("seg")
    
    for seg in segs:
        l = seg.childNodes[0].data
        l = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', l)
        sentence = l.split(' ')
        if len(sentence) > maxn:
            maxn = len(sentence)
        words.append(sentence)

    len1 = len(words)
    for i in range(len1):
        words_one = ['\t'] + words[i] + ['\n']
        len_word = len(words_one)
#         if len_word <= maxn:
#             for k in range(maxn-len_word):
#                 words_one.append("#")
        x_test.append(words_one)

    return x_test

In [14]:
def vec(n):
    return [(1 if i==n else 0) for i in range(INPUT_FEATURE_LENGTH)]

In [15]:
def vec_y(n):
    return [(1.0 if i==n else 0.0) for i in range(OUTPUT_FEATURE_LENGTH)]

In [16]:
x_dev = load_test('dev/newstest2015-deen-src.de.sgm')
y_dev = load_test('dev/newstest2015-deen-ref.en.sgm')

print(x_dev[1])
LEN_DEV = len(x_dev)

print(np.array(x_dev).shape,flush=True)
print(np.array(y_dev).shape,flush=True)

['\t', 'Indiens', 'neuer', 'Premierminister', 'Narendra', 'Modi', 'trifft', 'bei', 'seinem', 'ersten', 'wichtigen', 'Auslandsbesuch', 'seit', 'seinem', 'Wahlsieg', 'im', 'Mai', 'seinen', 'japanischen', 'Amtskollegen', 'Shinzo', 'Abe', 'in', 'Toko', ',', 'um', 'wirtschaftliche', 'und', 'sicherheitspolitische', 'Beziehungen', 'zu', 'besprechen', '.', '', '\n']
(2169,)
(2169,)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [17]:
encoder_input_dev = np.zeros((LEN_DEV,INUPT_LENGTH,INPUT_FEATURE_LENGTH))
decoder_input_dev = np.zeros((LEN_DEV,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))
decoder_output_dev = np.zeros((LEN_DEV,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH))

In [18]:
for seq_index,seq in enumerate(x_dev):
    for char_index, char in enumerate(seq):
        if char in input_dict and char_index < INUPT_LENGTH:
            encoder_input_dev[seq_index,char_index,input_dict[char]] = 1

In [19]:
for seq_index,seq in enumerate(y_dev):
    for char_index,char in enumerate(seq):
        if char in target_dict and char_index < OUTPUT_LENGTH:
            decoder_input_dev[seq_index,char_index,target_dict[char]] = 1.0
            if char_index > 0:
                decoder_output_dev[seq_index,char_index-1,target_dict[char]] = 1.0

In [20]:
# encoder_dev = []
# for i in x_dev:
#     line = []
#     for j in i:
#         if j in input_dict:
#             line.append(vec(input_dict[j]))
#     while len(line) < INUPT_LENGTH:
#         line.append(vec(-1))
#     encoder_dev.append(line)
    
# print(np.array(encoder_dev).shape)

In [21]:
# decoder_dev = []
# for i in y_dev:
#     line = []
#     for j in i:
#         if j in target_dict:
#             line.append(vec_y(target_dict[j]))
#     while len(line) < OUTPUT_LENGTH:
#         line.append(vec_y(-1))
#     decoder_dev.append(line)
    
# print(np.array(decoder_dev).shape)

In [22]:
# print(np.array(encoder_input).shape)
# print(np.array(decoder_input).shape)
# encoder_dev = np.array(encoder_dev)
# decoder_dev = np.array(decoder_dev)

# 创建模型

In [23]:
model_train, encoder_infer, decoder_infer = create_model(INPUT_FEATURE_LENGTH, OUTPUT_FEATURE_LENGTH, N_UNITS)

In [24]:
#查看模型结构
plot_model(to_file='model.png',model=model_train,show_shapes=True)
plot_model(to_file='encoder.png',model=encoder_infer,show_shapes=True)
plot_model(to_file='decoder.png',model=decoder_infer,show_shapes=True)

In [25]:
model_train.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [26]:
model_train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 6077)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 4988)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 6486016     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  5370880     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [27]:
encoder_infer.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 6077)        0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 6486016   
Total params: 6,486,016
Trainable params: 6,486,016
Non-trainable params: 0
_________________________________________________________________


In [28]:
decoder_infer.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 4988)   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  5370880     input_2[0][0]                    
                                                                 input_3[0][0]                    
          

# 模型训练

In [29]:
model_train.fit([encoder_input,decoder_input],decoder_output,batch_size=BATCH_SIZE,epochs=EPOCH,validation_data=([encoder_input_dev,decoder_input_dev],decoder_output_dev))

Train on 1024 samples, validate on 2169 samples
Epoch 1/10
1024/1024 [==============================] - 270s 263ms/step - loss: 1.3476 - val_loss: 0.7630
Epoch 2/10
1024/1024 [==============================] - 280s 274ms/step - loss: 1.2661 - val_loss: 0.8090
Epoch 3/10
1024/1024 [==============================] - 284s 277ms/step - loss: 1.2485 - val_loss: 0.7602
Epoch 4/10
1024/1024 [==============================] - 293s 286ms/step - loss: 1.2283 - val_loss: 0.7652
Epoch 5/10
1024/1024 [==============================] - 291s 284ms/step - loss: 1.2188 - val_loss: 0.7640
Epoch 6/10
1024/1024 [==============================] - 290s 283ms/step - loss: 1.2245 - val_loss: 0.7610
Epoch 7/10
1024/1024 [==============================] - 289s 282ms/step - loss: 1.1954 - val_loss: 0.7611
Epoch 8/10
1024/1024 [==============================] - 291s 284ms/step - loss: 1.1830 - val_loss: 0.7629
Epoch 9/10
1024/1024 [==============================] - 294s 287ms/step - loss: 1.1704 - val_loss: 0.761

# 测试集

In [30]:
x_test = load_test('test/newstest2016-deen-src.de.sgm')
y_test = load_test('test/newstest2016-deen-ref.en.sgm')

print(x_test[1])

print(np.array(x_test).shape,flush=True)
print(np.array(y_test).shape,flush=True)

['\t', 'Das', 'Verhältnis', 'zwischen', 'Obama', 'und', 'Netanyahu', 'ist', 'nicht', 'gerade', 'freundschaftlich', '.', '', '\n']
(2999,)
(2999,)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


In [31]:
# encoder_x_test = []
# for i in x_test:
#     line = []
#     for j in i:
#         if j in input_dict:
#             line.append(vec(input_dict[j]))
#     while len(line) < INUPT_LENGTH:
#         line.append(vec(-1))
#     encoder_x_test.append(line)
    
# print(np.array(encoder_x_test).shape)

In [32]:
LEN_TEST = len(x_test)
encoder_input_test = np.zeros((LEN_TEST,INUPT_LENGTH,INPUT_FEATURE_LENGTH))

for seq_index,seq in enumerate(x_test):
    for char_index, char in enumerate(seq):
        if char in input_dict and char_index < INUPT_LENGTH:
            encoder_input_test[seq_index,char_index,input_dict[char]] = 1

# 预测序列

In [33]:
def predict_it(source,encoder_inference, decoder_inference, n_steps, features):
    #先通过推理encoder获得预测输入序列的隐状态
    state = encoder_inference.predict(source)
    #第一个字符'\t',为起始标志
    predict_seq = np.zeros((1,1,features))
    predict_seq[0,0,target_dict['\t']] = 1

    output = ''
    #开始对encoder获得的隐状态进行推理
    #每次循环用上次预测的字符作为输入来预测下一次的字符，直到预测出了终止符
    for i in range(n_steps):#n_steps为句子最大长度
        #给decoder输入上一个时刻的h,c隐状态，以及上一次的预测字符predict_seq
        yhat,h,c = decoder_inference.predict([predict_seq]+state)
        #注意，这里的yhat为Dense之后输出的结果，因此与h不同
        char_index = np.argmax(yhat[0,-1,:])
        char = target_dict_reverse[char_index]
        output += char
        output += ' '
        state = [h,c]#本次状态做为下一次的初始状态继续传递
        predict_seq = np.zeros((1,1,features))
        predict_seq[0,0,char_index] = 1
        if char == '\n':#预测到了终止符则停下来
            break
    return output

In [34]:
mini = 0
maxi = LEN_TEST
y_predict = []

for i in range(mini,maxi):
    test = encoder_input_test[i:i+1,:,:]#i:i+1保持数组是三维
#     print(test)
    out = predict_it(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
    y_predict.append(out)
#     print(input_texts[i],'\n---\n',target_texts[i],'\n---\n',out)
    print(x_test[i])
    print(out)

['\t', 'Obama', 'empfängt', 'Netanyahu', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the 
['\t', 'Das', 'Verhältnis', 'zwischen', 'Obama', 'und', 'Netanyahu', 'ist', 'nicht', 'gerade', 'freundschaftlich', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the 
[

['\t', 'Es', 'gab', 'keinen', 'Hinweis', 'darauf', ',', 'dass', 'Lamb', ',', 'der', 'in', 'zwei', 'Online-Kursen', 'für', 'die', 'Delta', 'State', 'University', 'in', 'Cleveland', ',', 'Mississippi', ',', 'unterrichtete', ',', 'bereits', '300', 'Meilen', 'zu', 'dem', 'Schulgelände', 'gereist', 'war', ',', 'wo', 'er', 'nach', 'Angaben', 'der', 'Polizei', 'einen', 'beliebten', 'Geschichtsprofessor', ',', 'Ethan', 'Schmidt', ',', 'an', 'der', 'Tür', 'zu', 'seinem', 'Büro', 'erschossen', 'und', 'getötet', 'hat', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the 
['\t', 'Delta', 'State', 'University', 'Polizeichef', 'Lynn'

['\t', 'Lamb', 'hatte', 'zuvor', 'die', 'Delta', 'State', 'University', 'um', 'eine', 'Beurlaubung', 'aus', 'gesundheitlichen', 'Gründen', 'gebeten', 'und', 'dabei', 'gesagt', ',', 'dass', 'er', 'irgendein', 'gesundheitliches', 'Problem', 'habe', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the 
['\t', 'In', 'diesem', 'Jahr', 'unterrichtete', 'er', 'nur', 'zwei', 'Online-Kurse', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , t

['\t', 'Sie', 'und', 'ihre', 'Freunde', 'waren', 'in', 'der', 'High', 'School', ',', 'und', 'er', 'war', 'der', 'coole', 'ältere', 'Kerl', ',', 'der', 'einige', 'Jahre', 'versuchte', ',', 'ihr', 'das', 'Gitarrenspiel', 'beizubringen', '.', '', '\n']
, , , the the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , 
['\t', 'Er', 'war', 'damals', 'der', 'Frauenschwarm', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the 

['\t', 'Mike', 'Shaffer', ',', 'ein', 'Barkeeper', 'und', 'manchmal', 'Unterhaltungs-Koordinator', 'bei', 'The', 'Julep', 'Room', ',', 'einer', 'schwach', 'beleuchteten', 'Bar', 'in', 'der', 'Nähe', 'von', 'Gautier', ',', 'sagte', ',', 'dass', 'er', 'Lamb', 'und', 'Prentiss', 'Night', 'in', 'der', 'Nacht', '5', '.', 'September', 'zum', 'letzten', 'Mal', 'sah', ',', 'nachdem', 'Lamb', 'sein', 'Gitarren-', 'und', 'Mundharmonikaspiel', 'beendet', 'vor', 'einem', 'spärlichen', 'Publikum', 'beendet', 'hatte', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the 
['\t', 'Es', 'gab', 'keine', 'Anzeichen', 'dafür', ',', 'dass', 

...

['\t', 'Stattdessen', 'ist', 'die', 'Mode', 'wieder', 'wichtig', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the 
['\t', 'In', 'den', 'Anfangszeiten', 'der', 'Plauener', 'Spitze', 'spielte', 'Mode', 'eine', 'größere', 'Rolle', ',', 'wurde', 'dann', 'durch', 'die', 'Heimtextilindustrie', 'verdrängt', '-', 'und', 'wird', 'jetzt', 'als', 'Nische', 'wiederentdeckt', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the t

['\t', 'Reinhardt', 'ergänzt:', '"Das', 'bezieht', 'sich', 'auch', 'auf', 'die', 'verwendeten', 'Materialien"', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the 
['\t', 'Diplomdesignerin', 'Ute', 'Schmidt', 'lehrt', 'an', 'der', 'Schneeberger', 'Außenstelle', 'der', 'Westsächsischen', 'Hochschule', 'Zwickau', '-', 'an', 'der', 'Fakultät', 'für', 'angewandte', 'Kunst', '-', 'und', 'ist', 'auf', 'Stickereien', 'und', 'Spitzen', 'spezialisiert', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the 

['\t', 'Legenden', 'soll', 'man', 'ja', 'nicht', 'anrühren', ',', 'aber', 'Disney', 'wagt', 'sich', 'trotzdem', 'an', 'einen', 'neuen', 'Film', 'seines', 'Klassikers', '"Mary', 'Poppins"', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the 
['\t', 'Dabei', 'geht', 'es', 'laut', '"Entertainment', 'Weekly"', 'allerdings', 'nicht', 'um', 'eine', 'Neuverfilmung', ',', 'sondern', 'um', 'eine', 'Art', 'Fortsetzung', ',', 'die', '20', 'Jahre', 'nach', 'dem', 'ersten', 'Film', 'mit', 'dem', 'magischen', 'Kindermädchen', 'spielt', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , t

['\t', 'Die', 'Regierung', 'wird', 'auch', 'Kontrollen', 'über', 'die', 'an', 'die', 'Unternehmen', 'übertragenen', 'Befugnisse', 'lockern', 'und', 'dafür', 'sorgen', ',', 'dass', 'Unternehmen', 'entscheiden', ',', 'wie', 'sie', 'ihre', 'Geschäfte', 'führen', '.', '', '\n']
, , , the the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , 
['\t', 'Das', 'Statement', 'sagte', 'auch', ',', 'dass', 'China', 'Grenzgebieten', 'erlauben', 'wird', ',', 'neue', 'Modelle', 'der', 'grenzüberschreitenden', 'Wirtschaftskooperation', 'und', 'neue', 'Mechanismen', 'zur', 'Förderung', 'des', 'regionalen', 'Wachstums', 'zu', 'erkunden', '.', '', '\n']
, , , the th

['\t', 'Nach', 'Verdoppelung', 'der', 'Grenzwerte', 'wären', 'dann', 'nur', 'noch', '14', ',', '5', 'Prozent', 'unverkäuflich', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the 
['\t', 'Im', 'Gegenzug', 'für', 'dieses', 'Lockerung', 'will', 'die', 'EU', 'laut', 'Foodwatch', 'die', 'Grenzwerte', 'bei', 'anderen', 'Fischen', 'von', 'derzeit', '0', ',', '5', 'Milligramm', 'auf', '0', ',', '1', 'Milligramm', 'Quecksilber', 'pro', 'Kilogramm', 'verschärfen', '.', '', '\n']
, , , the the the the the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the , , the the ,

In [35]:
# from nltk.translate.bleu_score import sentence_bleu
# bi = []

# for i in range(mini,maxi):
#     bi.append(sentence_bleu(y_predict[i], target_texts[i]))
# print(np.array(bi).mean())

In [36]:
# mini = 0
# maxi = len(x_test)
# y_predict = []

# for i in range(mini,maxi):
# #     test = []
# #     test.append(encoder_x_test[i])#i:i+1保持数组是三维
# #     print(i)
#     test = []
#     test.append(encoder_x_test[i:i+1])
# #     print(test)
#     out = predict_it(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
#     y_predict.append(out)
# #     print(input_texts[i],'\n---\n',target_texts[i],'\n---\n',out)
#     print(out)

In [37]:
from nltk.translate.bleu_score import sentence_bleu

bi = []

for i in range(mini,maxi):
    bi.append(sentence_bleu(y_predict[i], y_test[i]))
print(np.array(bi).mean())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram o

4.014931229205878e-232
